

---
--- 
# MCCOY
## Millimiter Characterization of Complex Organics in Young stellar objects
--- 
---
#### Written and updated by Carlos E. Muñoz-Romero (2020)
![caption](imgs/mccoy.png)


#### Import dependencies

In [13]:
import dynesty
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sig

from dynesty import plotting as dyplot
from dynesty import utils as dyfunc
from numpy import random

from lmfit import Model, Parameters
from scipy.stats import norm, truncnorm
%matplotlib qt5

#### Function definitions

In [71]:
'''APPLY REDSHIFT TO SPECTRA'''
def redshift(spectrum, velocity):
    z = np.sqrt( (1 + (velocity/c)) / (1 - (velocity/c)) ) - 1
    return spectrum/(1+z)

'''CONVERT FREQUENCY TO VELOCITY IN KM/S'''
def freq_to_vel(spectrum, reference_frequency):
    return (np.array((c*reference_frequency - c*spectrum) / reference_frequency) )/1000

'''IDENTIFY EMISSION LINES'''
def identify(frequencies, velocities, intensities, catalog):
    
    peaks = sig.find_peaks(intensities)[0]
    peak_intensities = intensities[peaks]
    peak_frequencies = frequencies[peaks]
    
    # TAKE ONLY PEAKS ABOVE 5 SIGMA
    peak_frequencies = peak_frequencies[peak_intensities>=5*rms]
    peak_intensities = peak_intensities[peak_intensities>=5*rms]
    peak_velocities = freq_to_vel(peak_frequencies, reference_frequency)

    detection = dict({"frequencies":[], 
                      "velocities":[], 
                      "catalog_frequencies":[],
                      "eup":[],
                      "Smu2":[],
                      "logaij":[],
                      "transition":[]})
    
    for i,peak in enumerate(peak_frequencies):
        for j,line in enumerate(catalog["frequencies"]):
            if abs(peak-line)<=0.2 and "F" not in catalog["transition"][j]:
                detection["frequencies"].append(peak_frequencies[i])
                detection["velocities"].append(peak_velocities[i])
                detection["catalog_frequencies"].append(catalog["frequencies"][j])
                detection["eup"].append(catalog["eup"][j])
                detection["logaij"].append(catalog["logaij"][j])
                detection["Smu2"].append(catalog["Smu2"][j])
                detection["transition"].append(catalog["transition"][j])
                
    return detection

'GENERATE SPECTRAL WINDOWS AROUND EACH DETECTED TRANSITION'
def generate_windows(velocities, intensities, detections):
    windows = []

    for det_velocity in detections["velocities"]:
        window = dict({"velocities":[],
                       "intensities":[]})
        v = velocities[abs(velocities-det_velocity)<window_size]
        i = intensities[abs(velocities-det_velocity)<window_size]
        window["velocities"] = v-det_velocity
        window["intensities"] = i
        windows.append(window)
        
    return windows

'IDENTIFY POINTS WHICH MAKE UP THE EMISSION LINE'

# Pick all monotonically decreasing neighbors of each detected transition as a line to fit.
def idlines(windows, detections):
    lines = []
    for i,window in enumerate(windows):
        neighbors = dict({"velocities":[],
                          "intensities":[],
                          "indices":[],
                          "line_idx":0})
        line_idx = np.where(window["velocities"]+detections["velocities"][i] == detections["velocities"][i])[0][0]
        neighbors['line_idx'] = line_idx
        # Left neighbors
        diff = 0
        i = 1
        current = line_idx
        while diff <= 0 and abs(window["velocities"][current-i])<line_FWHM*5:
            current = line_idx-i
            neighbors["velocities"].append(window["velocities"][current])
            neighbors["intensities"].append(window["intensities"][current])
            neighbors["indices"].append(current)
            diff = window["intensities"][current-1]-window["intensities"][current]
            i+=1
        # Right neighbors
        diff = 0
        i = 0
        current = line_idx
        while diff <= 0 and abs(window["velocities"][current+i])<line_FWHM*5:
            current = line_idx+i
            neighbors["velocities"].append(window["velocities"][current])
            neighbors["intensities"].append(window["intensities"][current])
            neighbors["indices"].append(current)
            diff = window["intensities"][current+1]-window["intensities"][current]
            i+=1
        lines.append(neighbors)
    return lines

def gaussian(x, amp, center, fwhm):
    sigma = fwhm/(2*np.sqrt(2*np.log(2)))
    return amp / (np.sqrt(2*np.pi) * sigma) * np.exp(-(x-center)**2 / (2*(sigma**2)))

def polynomial_1d(x, m, b):
    return m*x + b

def emission_line_fitter(x, amp, center, fwhm, m, b):
    return gaussian(x, amp, center, fwhm)*is_line + polynomial_1d(x, m, b)

def emission_line(x, amp, center, fwhm, m, b):
    return gaussian(x, amp, center, fwhm) + polynomial_1d(x, m, b)

def emission_lmfit(x, y, is_line):
    
    emission_y = y*is_line
    gmodel = Model(emission_line_fitter)
    params = Parameters()
    params.add_many(('amp', 0.025, True, 0, 1, None, None),
                    ('center', 0, False, -1, 1, None, None),
                    ('fwhm', 0.5, True, 0, 3, None, None),
                    ('m', 0, True, -1, 1, None, None),
                    ('b', 0, True, -0.02, 0.1, None, None))
    
    result = gmodel.fit(y, params, x=x)
    return result

def rotdiag(x, log10Ntot, T, tau):
    Ntot = 10**log10Ntot
    C = tau / (1 - np.exp(-tau))
    return np.log(Ntot) - np.log(Q(T)) - np.log(C) - (1/T)*x

def rotdiag_lmfit(x, y):
    
    rmodel = Model(rotdiag)
    params = Parameters()
    params.add_many(('log10Ntot', 13, True, 8, 14, None, None),
                    ('T', 20, True, 1, 150, None, None),
                    ('tau', 1e-4, True, 1e-10, 1, None, None))
    
    result = rmodel.fit(y, params, x=x)
    return result


#### User input
The pipeline is compatible with tab-separated spectroscopy files generated by the NRAO splatalogue database, with frequency in MHz. (https://www.cv.nrao.edu/php/splat/). It is necessary to add extra columns to this file to estimate the rotational partition function via interpolation, if desired.

In [7]:
'SPLATALOGUE FILE'
molecule_name = "CH3CN"
catalog_file = "./splatalogue/CH3CN_catalog.tsv"
spectrum_file = "./181_WSW_FTS200_3mm_average_data_Tmb.dat"
reference_frequency = 93750.7299 
'ESTIMATE OF FULL WIDTH AT HALF MAXIMUM OF EMISSION LINES IN [km/s]'
line_FWHM = 0.5
'VELOCITY OF THE SOURCE IN [km/s]'
v_lsr = 0
'ROOT MEAN SQUARE OF INTENSITY DATA IN [K]'
rms = 5e-3
'SIZE OF SPECTRAL WINDOW TO USE AROUND EACH DETECTED TRANSITION IN [km/s]'
window_size = 25
'CONSTANTS'
c =  2.998*1e8 # m/s
k = 1.3807 * 1e-16 # erg/K
toHz = 1e6

#### Load data and transform to velocity units (km/s)

In [8]:
spectrum = np.loadtxt(spectrum_file)
frequencies = redshift(spectrum[:,0], v_lsr)
velocities = freq_to_vel(frequencies, reference_frequency)
intensities = spectrum[:,1]
catalog_dataframe = pd.read_csv(catalog_file, delimiter="\t", header=0, index_col=False)
catalog = dict({"frequencies":catalog_dataframe["Freq-MHz(rest frame,redshifted)"], 
                "velocities":freq_to_vel(catalog_dataframe["Freq-MHz(rest frame,redshifted)"], reference_frequency),
                "Smu2":catalog_dataframe["S<sub>ij</sub>&#956;<sup>2</sup> (D<sup>2</sup>)"],
                "eup":catalog_dataframe["E_U (K)"],
                "logaij":catalog_dataframe["Log<sub>10</sub> (A<sub>ij</sub>)"],
                "transition":catalog_dataframe["Resolved QNs"],
                "Qrot":catalog_dataframe["Q"],
                "Trot":catalog_dataframe["T"],
                "linelist":catalog_dataframe["Linelist"]})

## Line Fitting

#### Find all emission lines

In [9]:
detections = identify(frequencies, velocities, intensities, catalog)


#### Define spectral windows and center around zero

In [10]:
windows = generate_windows(velocities, intensities, detections)
lines = idlines(windows, detections)

In [11]:
for window in windows:
    plt.figure()
    plt.step(window["velocities"], window["intensities"])

#### Perform initial fit of emission lines via non-linear least-squares minimization. Then, estimate uncertainties via nested sampling of the posterior distribution, using truncated normal priors centered around the lmfit results

In [14]:
def loglike(theta):
    amp, fwhm, lnf = theta
    m = lmfit_m
    b = lmfit_b
    center = 0 
   
    model = emission_line(x, amp, center, fwhm, m, b)
    inv_sigma2 = 1.0 / (yerr**2 + model**2 * np.exp(2 * lnf))
    return -0.5 * (np.sum((y-model)**2 * inv_sigma2 - np.log(inv_sigma2)))

line_results_dynesty = []
line_results_lmfit = []

for i in range(len(lines)):
    is_line = np.zeros(len(windows[i]["velocities"]))
    is_line[lines[i]["indices"]] = 1

    x = windows[i]["velocities"]
    y = windows[i]["intensities"]

    lmfit_result = emission_lmfit(x, y, is_line)
    line_results_lmfit.append(lmfit_result)
    
    lmfit_amp = lmfit_result.best_values["amp"]
    lmfit_center = lmfit_result.best_values["center"]
    lmfit_fwhm = lmfit_result.best_values["fwhm"]
    lmfit_m = lmfit_result.best_values["m"]
    lmfit_b = lmfit_result.best_values["b"]
    
    x = x[is_line==1]
    y = y[is_line==1]
    y = [k if k>0 else 0 for k in y]
    yerr = random.randn(len(y))*0.001
    
    def prior_transform(utheta):
        uamp, ufwhm, ulnf = utheta
        # Truncated Normal
        mean_amp , s_amp  = lmfit_amp, 0.2*lmfit_amp  # mean and standard deviation
        low_amp , high_amp  = 0, 2*lmfit_amp  
        low_n_amp, high_n_amp = (low_amp - mean_amp ) / s_amp , (high_amp - s_amp ) / s_amp  # standardize
        amp = truncnorm.ppf(uamp, low_n_amp, high_n_amp, loc=mean_amp, scale=s_amp)
        # Truncated Normal
        mean_fwhm , s_fwhm  = lmfit_fwhm, 0.2*lmfit_fwhm  # mean and standard deviation
        low_fwhm , high_fwhm  = 0, 2*lmfit_fwhm
        low_n_fwhm, high_n_fwhm = (low_fwhm - mean_fwhm ) / s_fwhm , (high_fwhm - s_fwhm ) / s_fwhm  # standardize
        fwhm = truncnorm.ppf(ufwhm, low_n_fwhm, high_n_fwhm, loc=mean_fwhm, scale=s_fwhm)
        
        lnf = 15. * ulnf - 10.
        return amp, fwhm, lnf
    
    dsampler = dynesty.DynamicNestedSampler(loglike, prior_transform, ndim=3,
                                            bound='multi', sample='auto')
    dsampler.run_nested(dlogz_init=0.01)
    dres = dsampler.results
    line_results_dynesty.append(dres)

22391it [05:42, 65.43it/s, batch: 14 | bound: 22 | nc: 1 | ncall: 218967 | eff(%): 10.226 | loglstar:  8.967 < 17.306 < 16.923 | logz: 13.975 +/-  0.074 | stop:  0.871]           
19047it [03:55, 80.90it/s, batch: 10 | bound: 28 | nc: 16 | ncall: 168396 | eff(%): 11.311 | loglstar: 19.571 < 21.574 < 21.226 | logz: 17.629 +/-  0.078 | stop:  0.883]          
19259it [04:21, 73.61it/s, batch: 11 | bound: 25 | nc: 17 | ncall: 187929 | eff(%): 10.248 | loglstar: 19.950 < 22.197 < 21.784 | logz: 18.680 +/-  0.076 | stop:  0.914]       
18910it [03:56, 80.11it/s, batch: 10 | bound: 26 | nc: 4 | ncall: 171511 | eff(%): 11.026 | loglstar: 19.549 < 21.647 < 21.229 | logz: 17.651 +/-  0.078 | stop:  0.939]           
21117it [04:28, 78.63it/s, batch: 11 | bound: 30 | nc: 17 | ncall: 193126 | eff(%): 10.934 | loglstar: 24.458 < 27.109 < 26.558 | logz: 23.139 +/-  0.073 | stop:  0.992]       
19128it [03:48, 83.75it/s, batch: 10 | bound: 26 | nc: 5 | ncall: 166363 | eff(%): 11.498 | loglstar: 23.8

In [15]:
fluxes = []    
widths = []    

flux_errors = []
widths_errors = []

for i,dres in enumerate(line_results_dynesty):
    
    x = windows[i]["velocities"]
    y = windows[i]["intensities"]
    # Extract sampling results.
    samples = dres.samples  # samples
    weights = np.exp(dres.logwt - dres.logz[-1])  # normalized weights
    
    lmfit_result = line_results_lmfit[i]
    lmfit_amp = lmfit_result.best_values["amp"]
    lmfit_center = lmfit_result.best_values["center"]
    lmfit_fwhm = lmfit_result.best_values["fwhm"]
    lmfit_m = lmfit_result.best_values["m"]
    lmfit_b = lmfit_result.best_values["b"]
    
    
    # Compute quantiles.
    quantiles = [dyfunc.quantile(samps, [0.1586, 0.5, 0.84135], weights=weights)
                 for samps in samples.T]
    
    dynesty_flux = quantiles[0][1]
    dynesty_fwhm = quantiles[1][1]
    
    fluxes.append(dynesty_flux)
    widths.append(dynesty_fwhm)
    flux_errors.append([dynesty_flux-quantiles[0][0], quantiles[0][2]-dynesty_flux])
    widths_errors.append([dynesty_fwhm-quantiles[1][0], quantiles[1][2]-dynesty_fwhm])
    
    plt.figure()
    window_linspace = np.linspace(min(windows[i]["velocities"]), max(windows[i]["velocities"]),10000)
    plt.step(x, y, color="gray")
    
    plt.plot(window_linspace, emission_line(window_linspace, lmfit_amp,
                                        lmfit_center, lmfit_fwhm, lmfit_m, lmfit_b),color="red")
    plt.plot(window_linspace, emission_line(window_linspace, dynesty_flux,
                                        lmfit_center, dynesty_fwhm, lmfit_m, lmfit_b),color="green")



In [18]:
labels = [r'$\int T dv$ ', r'FWHM ',r'$\ln f$']
for dres in line_results_dynesty:    
    #fig, axes = dyplot.traceplot(dres, labels=labels, title_fmt=(".4f"),
                                 #trace_cmap='inferno', post_color="black", label_kwargs=dict({"fontsize":15}))
    fig, axes = dyplot.cornerplot(dres, color='dodgerblue', show_titles=True, title_fmt=(".4f"), labels=labels)


## Rotation Diagram

In [72]:
strengths = np.array(detections["Smu2"]) * (1e-18)**2 # from D^2 to (statC cm)^2
fluxes = np.array(fluxes)
fluxes_cgs = fluxes*100000 # in cm/s
widths = np.array(widths)
flux_errors = np.array(flux_errors).reshape(len(lines),2)
flux_errors_cgs = np.array(flux_errors).reshape(6,2)*100000 
widths_errors = np.array(widths_errors).reshape(6,2)
# Upper-level populations Nu/gu
nugu = (3*k*fluxes_cgs)/(8*(np.pi**3)*np.array(detections["catalog_frequencies"])*toHz*strengths)
lnnugu = np.log(nugu)
lnnugu_err = flux_errors / np.array([[f,f] for f in fluxes]).reshape(len(lines),2)


In [73]:
Q_array = catalog["Qrot"]
T_array  = catalog["Trot"]
Q_array = Q_array[Q_array == Q_array]
T_array = T_array[T_array == T_array]

'FIT A POLYNOMIAL TO THE PARTITION FUNCTION DATA TO INTERPOLATE'
Qfit = np.polyfit(T_array, Q_array, 3)
Q = np.poly1d(Qfit)

x = np.array(detections["eup"])
y = lnnugu
yerr = lnnugu_err.T

lmfit_result = rotdiag_lmfit(x, y)    
lmfit_T = lmfit_result.best_values["T"]
lmfit_log10Ntot = lmfit_result.best_values["log10Ntot"]
lmfit_tau = lmfit_result.best_values["tau"]
lmfit_result

In [78]:
# log-likelihood
def loglike(theta):
    
    T, log10Ntot, tau, lnf = theta
    model = rotdiag(x,log10Ntot,T, tau)
    inv_sigma2 = 1.0 / (yerr**2 + model**2 * np.exp(2 * lnf))
    return -0.5 * (np.sum((y-model)**2 * inv_sigma2 - np.log(inv_sigma2)))

# prior transform
def prior_transform(utheta):
    uT, ulog10Ntot, utau, ulnf = utheta
    
    log10Ntot = 5*ulog10Ntot + 8
    # Truncated Normal
    mean_T , s_T  = lmfit_T, 0.2*lmfit_T  # mean and standard deviation
    low_T , high_T  = 0.5*lmfit_T, 1.5*lmfit_T
    low_n_T, high_n_T = (low_T - mean_T ) / s_T , (high_T - s_T ) / s_T  # standardize
    T = truncnorm.ppf(uT, low_n_T, high_n_T, loc=mean_T, scale=s_T)
    # Truncated Normal
    mean_log10Ntot , s_log10Ntot  = lmfit_log10Ntot, 0.2*lmfit_log10Ntot  # mean and standard deviation
    low_log10Ntot , high_log10Ntot  = 0.5*lmfit_log10Ntot, 1.5*lmfit_log10Ntot
    low_n_log10Ntot, high_n_log10Ntot = (low_log10Ntot - mean_log10Ntot ) / s_log10Ntot , (high_log10Ntot - s_log10Ntot ) / s_log10Ntot  # standardize
    log10Ntot = truncnorm.ppf(ulog10Ntot, low_n_log10Ntot, high_n_log10Ntot, loc=mean_log10Ntot, scale=s_log10Ntot)
    # Truncated Normal
    mean_tau , s_tau  = lmfit_tau, 0.1*lmfit_tau  # mean and standard deviation
    low_tau , high_tau  = 0.3*lmfit_tau, 1
    low_n_tau, high_n_tau = (low_tau - mean_tau ) / s_tau , (high_tau - s_tau ) / s_tau  # standardize
    tau = truncnorm.ppf(utau, low_n_tau, high_n_tau, loc=mean_tau, scale=s_tau)   
    lnf = 21 * ulnf - 20.
    return T, log10Ntot, tau, lnf

dsampler = dynesty.DynamicNestedSampler(loglike, prior_transform, ndim=4, bound='multi', sample='auto')
dsampler.run_nested(dlogz_init=0.01)
rotdiag_res = dsampler.results

27681it [05:17, 87.31it/s, batch: 12 | bound: 92 | nc: 1 | ncall: 105840 | eff(%): 26.154 | loglstar:   -inf <  7.611 <  7.058 | logz: -0.124 +/-  0.143 | stop:  0.821]            


In [81]:
# Extract sampling results.
samples = rotdiag_res.samples  # samples
weights = np.exp(rotdiag_res.logwt - rotdiag_res.logz[-1])  # normalized weights
quantiles = [dyfunc.quantile(samps, [0.025, 0.1586, 0.5, 0.84135, 0.975], weights=weights)
                 for samps in samples.T]

labels = [r'T$_{rot}$ ', r'log$_{10}$(N$_{tot}$)',r'$\tau$']
fig, axes = dyplot.cornerplot(rotdiag_res, color='brown', show_titles=True, dims=[0,1,2],
                              title_fmt=(".4f"), labels=labels, label_kwargs=dict({"fontsize":15}))

In [39]:
plt.errorbar(x,y,yerr=lnnugu_err.T, fmt="o",color="black",capsize=4)
alphas = weights/max(weights)*0.1
eulinspace = np.linspace(0,max(x)+10)
for i,sample in enumerate(rotdiag_res.samples):
    plt.plot(eulinspace, rotdiag(eulinspace, sample[1], sample[0], sample[2]),alpha=alphas[i],
             color="dodgerblue", linewidth=0.1)
plt.plot(eulinspace, rotdiag(eulinspace,quantiles[1][2],quantiles[0][2],quantiles[2][2]),
         color="black",linestyle="--",linewidth=2)


In [42]:
np.save("./fits/{}_lines_dynesty".format(molecule_name), line_results_dynesty)
np.save("./fits/{}_lines_lmfit".format(molecule_name), line_results_lmfit)
np.save("./fits/{}_rotdiag_lmfit".format(molecule_name), lmfit_result)
np.save("./fits/{}_rotdiag_dynesty".format(molecule_name), dres)
np.save("./fits/{}_detection_info".format(molecule_name), detections)